In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras
import tensorflow
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-05-13 13:39:04.990842: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 13:39:04.990952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 13:39:05.106176: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
labels = [
    'airplane',
'automobile',
'bird',
'cat',
'deer',
'dog',
'frog',
'horse',
'ship',
'truck'
]

In [3]:
class_labels_cifar10 = {index: val for index, val in enumerate(labels)}

In [4]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data() 

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 13s 0us/step


In [5]:
bat_size = 32
image_size = 32
image_channel = 3



In [6]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
train_labels_ohe = enc.fit_transform(train_labels).toarray().astype(int)
test_labels_ohe = enc.transform(test_labels).toarray().astype(int)

In [7]:
from sklearn.model_selection import train_test_split 
x_te, x_val, y_te, y_val = train_test_split(test_images, test_labels_ohe, test_size=0.3, random_state=21)

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range = 15,
                                    horizontal_flip = True,
                                    zoom_range = 0.2,
                                    shear_range = 0.1,
                                    fill_mode = 'reflect',
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1)

test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train_generator = train_datagen.flow(train_images, train_labels_ohe,
                                                    batch_size = bat_size
                                                   )
val_generator = test_datagen.flow(x_val,  y_val,
                                                 batch_size = bat_size,
                                                 shuffle=False
                                                )

test_generator = test_datagen.flow(x_te,  y_te,
                                                  batch_size = bat_size,
                                                  shuffle=False
                                                 )

In [10]:
from tensorflow.keras import applications

resnet = applications.ResNet101(
    include_top=True,
    weights= None,
    input_tensor=None,
    input_shape= train_images.shape[1:],
    classes=10,
    classifier_activation='softmax'
)

In [11]:
resnet.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [52]:
history_resnet = resnet.fit(train_generator, epochs=5, batch_size=32, validation_data = val_generator) # ~90 epochs

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 86s 55ms/step - acc: 0.8803 - loss: 0.3493 - val_acc: 0.7843 - val_loss: 0.7685
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 86s 55ms/step - acc: 0.8694 - loss: 0.3747 - val_acc: 0.6687 - val_loss: 1.6219
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 86s 55ms/step - acc: 0.7925 - loss: 0.6047 - val_acc: 0.8133 - val_loss: 1.2532
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 86s 55ms/step - acc: 0.8670 - loss: 0.3834 - val_acc: 0.8223 - val_loss: 1.5065
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 87s 55ms/step - acc: 0.8890 - loss: 0.3250 - val_acc: 0.8217 - val_loss: 1.3067


In [53]:
loss, acc = resnet.evaluate(test_generator)
print("desnet model, accuracy: {:5.2f}%".format(100 * acc))

219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - acc: 0.8190 - loss: 1.3334
desnet model, accuracy: 81.76%


In [17]:
mobilenet = applications.MobileNet(
    input_shape=train_images.shape[1:],
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights= None,
    input_tensor=None,
    pooling=None,
    classes=10,
    classifier_activation="softmax",
    
)

In [18]:
mobilenet.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [35]:
history_mobilenet = mobilenet.fit(train_generator, epochs=2, batch_size=32, validation_data = val_generator) # ~90 epoch

Epoch 1/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 37s 24ms/step - acc: 0.8807 - loss: 0.3398 - val_acc: 0.8380 - val_loss: 0.5035
Epoch 2/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 37s 23ms/step - acc: 0.8848 - loss: 0.3397 - val_acc: 0.8573 - val_loss: 0.4334


In [36]:
loss, acc = mobilenet.evaluate(test_generator)
print("mobilenet model, accuracy: {:5.2f}%".format(100 * acc))

219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - acc: 0.8499 - loss: 0.4718
mobilenet model, accuracy: 85.01%


In [60]:
from keras import Model, layers
from tensorflow.keras.initializers import RandomNormal, Constant

net = {}
net['input'] = layers.Input((32, 32, 3))

net['conv_1'] = layers.Conv2D(256, kernel_size=(3, 3), padding='same')(net['input'])
net['batch_1'] = layers.BatchNormalization()(net['conv_1'])
net['activ_1'] = layers.Activation('relu')(net['batch_1'])
net['conv_2'] = layers.Conv2D(256, kernel_size=(3, 3), padding='same')(net['activ_1'])
net['batch_2'] = layers.BatchNormalization()(net['conv_2'])
net['activ_2'] = layers.Activation('relu')(net['batch_2'])
net['pool_1'] = layers.MaxPooling2D(pool_size=(2,2))(net['activ_2'])
net['dropout_1'] = layers.Dropout(0.2)(net['pool_1'])



net['conv_3'] = layers.Conv2D(512, kernel_size=(3, 3), padding='same')(net['dropout_1'])
net['batch_3'] = layers.BatchNormalization()(net['conv_3'])
net['activ_3'] = layers.Activation('relu')(net['batch_3'])
net['conv_4'] = layers.Conv2D(512, kernel_size=(3, 3), padding='same')(net['activ_3'])
net['batch_4'] = layers.BatchNormalization()(net['conv_4'])
net['activ_4'] = layers.Activation('relu')(net['batch_4'])
net['pool_2'] = layers.MaxPooling2D(pool_size=(2,2))(net['activ_4'])
net['dropout_2'] = layers.Dropout(0.2)(net['pool_2'])



net['conv_5'] = layers.Conv2D(512, kernel_size=(3, 3), padding='same')(net['dropout_2'])
net['batch_5'] = layers.BatchNormalization()(net['conv_5'])
net['activ_5'] = layers.Activation('relu')(net['batch_5'])
net['conv_5'] = layers.Conv2D(512, kernel_size=(3, 3), padding='same')(net['activ_5'])
net['batch_6'] = layers.BatchNormalization()(net['conv_5'])
net['activ_6'] = layers.Activation('relu')(net['batch_6'])
net['pool_3'] = layers.MaxPooling2D(pool_size=(2,2))(net['activ_6'])
net['dropout_3'] = layers.Dropout(0.2)(net['pool_3'])

net['conv_7'] = layers.Conv2D(512, kernel_size=(3, 3), padding='same')(net['dropout_3'])
net['batch_7'] = layers.BatchNormalization()(net['conv_7'])
net['activ_7'] = layers.Activation('relu')(net['batch_7'])
net['conv_8'] = layers.Conv2D(512, kernel_size=(3, 3), padding='same')(net['activ_7'])
net['batch_8'] = layers.BatchNormalization()(net['conv_8'])
net['activ_8'] = layers.Activation('relu')(net['batch_8'])
net['pool_4'] = layers.MaxPooling2D(pool_size=(2,2))(net['activ_8'])
net['dropout_4'] = layers.Dropout(0.2)(net['pool_4'])

net['flat'] = layers.Flatten()(net['dropout_4'])
net['dense_1'] = layers.Dense(1024)(net['flat'])
net['activ_7'] = layers.Activation('relu')(net['dense_1'])
net['dropout_4'] = layers.Dropout(0.2)(net['activ_7'])
net['batch_7'] = layers.BatchNormalization()(net['dropout_4'])
net['output'] = layers.Dense(10, activation='softmax')(net['batch_7'])

model = Model(net['input'], net['output'])

In [61]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [62]:
history_model = model.fit(train_generator, epochs=12, batch_size=32, validation_data = val_generator) # 12 epochs

Epoch 1/12
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 81s 41ms/step - acc: 0.3227 - loss: 1.9101 - val_acc: 0.5390 - val_loss: 1.4087
Epoch 2/12
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 53s 34ms/step - acc: 0.5725 - loss: 1.2014 - val_acc: 0.6267 - val_loss: 1.2029
Epoch 3/12
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 53s 34ms/step - acc: 0.6797 - loss: 0.9285 - val_acc: 0.7310 - val_loss: 0.7552
Epoch 4/12
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 53s 34ms/step - acc: 0.7265 - loss: 0.7992 - val_acc: 0.6237 - val_loss: 1.2223
Epoch 5/12
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 53s 34ms/step - acc: 0.7588 - loss: 0.7063 - val_acc: 0.7443 - val_loss: 0.8123
Epoch 6/12
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 53s 34ms/step - acc: 0.7865 - loss: 0.6290 - val_acc: 0.8203 - val_loss: 0.5158
Epoch 7/12
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 53s 34ms/step - acc: 0.8085 - loss: 0.5636 - val_acc: 0.7893 - val_loss: 0.6747
Epoch 8/12
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 53s 34ms/step - acc: 0.8199 - loss: 0.5248 - val_acc: 0.8180 - val_loss: 0.5645
Epoch 9/12
1563/1563 ━━━

In [63]:
loss, acc = model.evaluate(test_generator)
print("Model, accuracy: {:5.2f}%".format(100 * acc))

219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - acc: 0.8652 - loss: 0.4033
Model, accuracy: 87.11%


# Average ensemble

In [100]:
from tensorflow.keras import layers, Model
import itertools


models = [resnet, mobilenet, model]
model_input = layers.Input(shape=(32, 32, 3))
model_outputs = [model(model_input) for model in models]
flattened_outputs = model_outputs
ensemble_output = layers.Average()(flattened_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output)

In [101]:
ensemble_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [102]:
loss, acc = ensemble_model.evaluate(test_generator)
print("Average_ensemble, accuracy: {:5.2f}%".format(100 * acc))

219/219 ━━━━━━━━━━━━━━━━━━━━ 22s 47ms/step - acc: 0.8851 - loss: 0.3495
Average_ensemble, accuracy: 88.69%


# Weighted ensemble

In [103]:
class WeightedAverageLayer(tensorflow.keras.layers.Layer):
    def __init__(self, w1, w2, w3, **kwargs):
        super(WeightedAverageLayer, self).__init__(**kwargs)
        self.w1 = w1
        self.w2 = w2
        self.w3 = w3

    def call(self, inputs):
        return self.w1 * inputs[0] + self.w2 * inputs[1] + self.w3 * inputs[2]

In [121]:
models = [resnet, mobilenet, model]
model_input = layers.Input(shape=(32, 32, 3))
model_outputs = [model(model_input) for model in models]
flattened_outputs = model_outputs
ensemble_output_w = WeightedAverageLayer(2/12, 3/12, 7/12)(flattened_outputs)
ensemble_model_w = Model(inputs=model_input, outputs=ensemble_output_w)

In [122]:
ensemble_model_w.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [123]:
print("Weighted_ensemble, accuracy: {:5.2f}%".format(100 * ensemble_model_w.evaluate(test_generator)[1]))

219/219 ━━━━━━━━━━━━━━━━━━━━ 22s 48ms/step - acc: 0.8845 - loss: 0.3379
Weighted_ensemble, accuracy: 88.87%


### Average_ensemble == 88.69% < 88.87% Weighted_ensemble

In [107]:
from tensorflow.keras.models import load_model

resnet.save('/kaggle/working/resnet_cifar10_a_w.hdf5')
resnet.save('/kaggle/working/resnet_cifar10_a_w.h5')

mobilenet.save('/kaggle/working/mobilenet_cifar10_a_w.hdf5')
mobilenet.save('/kaggle/working/mobilenet_cifar10_a_w.h5')

model.save('/kaggle/working/model_cifar10_a_w.hdf5')
model.save('/kaggle/working/model_cifar10_a_w.h5')